In [1]:
import pandas as pd
import ast
import os

In [2]:
# ---------------------------------------------------------
# 1. LOAD DATA WITH PIPE DELIMITER
# ---------------------------------------------------------
df = pd.read_csv("/home/uwe/research/data.csv/data.csv", sep="|", engine="python", on_bad_lines="skip")

In [3]:
# ---------------------------------------------------------
# 2. CLEAN COLUMN NAMES
# ---------------------------------------------------------

df.columns = df.columns.str.strip()
df.columns

Index(['name', 'slug', 'full_name', 'type', 'publisher', 'description',
       'source', 'popularity', 'star_count', 'pull_count', 'filter_type',
       'certification_status', 'categories', 'operating_systems',
       'architectures', 'logo_url', 'created_at', 'updated_at'],
      dtype='object')

In [4]:
# ---------------------------------------------------------
# 3. ENFORCE DATA TYPES
# ---------------------------------------------------------

# String columns
string_cols = [
    "name", "slug", "full_name", "type", "publisher",
    "description", "source", "filter_type",
    "certification_status", "logo_url"
]

for col in string_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

df[col]

0       https://mcrflowpublic.blob.core.windows.net/as...
1       https://mcrflowpublic.blob.core.windows.net/as...
2       https://mcrflowpublic.blob.core.windows.net/as...
3       https://raw.githubusercontent.com/dotnet/brand...
4       https://raw.githubusercontent.com/dotnet/brand...
                              ...                        
5660                                                  nan
5661                                                  nan
5662                                                  nan
5663                                                  nan
5664                                                  nan
Name: logo_url, Length: 5665, dtype: object

In [5]:
# Integer columns
int_cols = ["popularity", "star_count", "pull_count"]

for col in int_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

df[col]

0       0
1       0
2       0
3       0
4       0
       ..
5660    0
5661    0
5662    0
5663    0
5664    0
Name: pull_count, Length: 5665, dtype: int64

In [6]:
# Datetime columns
date_cols = ["created_at", "updated_at"]

for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

df[col]

0      2022-04-13 02:28:59.163440+00:00
1      2022-04-13 02:28:59.054090+00:00
2      2022-04-13 02:28:58.981136+00:00
3      2022-04-13 02:28:55.315394+00:00
4      2022-04-13 02:28:55.493136+00:00
                     ...               
5660   2017-11-09 05:32:16.340209+00:00
5661   2022-01-06 14:08:18.228185+00:00
5662   2022-04-04 17:25:16.988374+00:00
5663   2021-06-16 02:00:01.126549+00:00
5664   2022-04-11 19:41:18.242566+00:00
Name: updated_at, Length: 5665, dtype: datetime64[ns, UTC]

In [7]:
# ---------------------------------------------------------
# 4. CLEAN ARRAY COLUMNS (convert strings → Python lists)
# ---------------------------------------------------------

array_cols = ["categories", "operating_systems", "architectures"]

def to_list(x):
    if pd.isna(x):
        return []
    try:
        # Try evaluating if it's a JSON-like string
        return ast.literal_eval(x)
    except:
        # Fallback: split by comma
        return [item.strip() for item in str(x).split(",") if item.strip()]

for col in array_cols:
    if col in df.columns:
        df[col] = df[col].apply(to_list)

df[col]

0         [x86-64]
1         [x86-64]
2         [x86-64]
3         [x86-64]
4         [x86-64]
           ...    
5660      [x86-64]
5661      [x86-64]
5662            []
5663      [x86-64]
5664    [, x86-64]
Name: architectures, Length: 5665, dtype: object

In [8]:
# ---------------------------------------------------------
# 5. REMOVE DUPLICATES & EMPTY ROWS
# ---------------------------------------------------------

for col in array_cols:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: str(x))

df.drop_duplicates(inplace=True)
df.dropna(how="all", inplace=True)

In [9]:
# =========================================================
# 6. SAVE CLEANED DATA
# =========================================================

output_path = "cleaned_data.csv"
df.to_csv(output_path, index=False)

print("[✓] Data cleaning completed.")
print("[✓] Cleaned file saved as:", output_path)

df.head()

[✓] Data cleaning completed.
[✓] Cleaned file saved as: cleaned_data.csv


,name,slug,full_name,type,publisher,description,source,popularity,star_count,pull_count,filter_type,certification_status,categories,operating_systems,architectures,logo_url,created_at,updated_at
0,Windows base OS images,microsoft-windows-base-os-images,Microsoft/microsoft-windows-base-os-images,image,Microsoft,Product family for all Windows base OS contain...,publisher,1134293415,0,0,nan,not_certified,['Base Images'],['Linux'],['x86-64'],https://mcrflowpublic.blob.core.windows.net/as...,2018-11-27 04:52:14.129981+00:00,2022-04-13 02:28:59.163440+00:00
1,Windows Server Core,microsoft-windows-servercore,Microsoft/microsoft-windows-servercore,image,Microsoft,The official Windows Server Core base image fo...,publisher,587010244,0,0,nan,not_certified,['Base Images'],['Linux'],['x86-64'],https://mcrflowpublic.blob.core.windows.net/as...,2018-11-30 21:33:55.584313+00:00,2022-04-13 02:28:59.054090+00:00
2,Nano Server,microsoft-windows-nanoserver,Microsoft/microsoft-windows-nanoserver,image,Microsoft,The official Nano Server base image for contai...,publisher,539146631,0,0,nan,not_certified,['Base Images'],['Linux'],['x86-64'],https://mcrflowpublic.blob.core.windows.net/as...,2018-11-29 17:26:14.311143+00:00,2022-04-13 02:28:58.981136+00:00
3,.NET Core,microsoft-dotnet-core,Microsoft/microsoft-dotnet-core,image,Microsoft,Official images for .NET Core 3.1 and ASP.NET ...,publisher,279330629,0,0,nan,not_certified,"['Application Frameworks', 'Programming Langua...",['Linux'],['x86-64'],https://raw.githubusercontent.com/dotnet/brand...,2019-02-26 21:31:35.258060+00:00,2022-04-13 02:28:55.315394+00:00
4,.NET,microsoft-dotnet,Microsoft/microsoft-dotnet,image,Microsoft,Official images for .NET and ASP.NET Core,publisher,258250089,0,0,nan,not_certified,"['Application Frameworks', 'Programming Langua...",['Linux'],['x86-64'],https://raw.githubusercontent.com/dotnet/brand...,2020-05-13 09:21:57.246732+00:00,2022-04-13 02:28:55.493136+00:00
